In [6]:
!pip install dash==0.29.0

     |████████████████████████████████| 95 kB 6.1 MB/s  eta 0:00:01
     |████████████████████████████████| 1.0 MB 40.8 MB/s eta 0:00:01
     |████████████████████████████████| 357 kB 39.4 MB/s eta 0:00:01
  Created wheel for dash: filename=dash-0.29.0-py3-none-any.whl size=30405 sha256=5ed73637ba54d2b8f61d34b0d64f7f8a3fbcad6126918eca0e662852fbacf395
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/16/91/54/1296b6d280c77eb28ff930d92ec78fd4332ca9af2917cca72f
  Created wheel for dash-renderer: filename=dash_renderer-1.9.1-py3-none-any.whl size=1014873 sha256=d6a2483d3d2243763323dac471495ab7e8394c5f24fe69ac9f7e198d1cc66c5e
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/52/2f/28/956ce9f7763c886d23f0082829bd726e5dacd6c708a5841c77
Successfully built dash dash-renderer


In [7]:
conda install dash dash-core-components dash-html-components dash-renderer -c conda-forge

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.9

  added / updated specs:
    - dash
    - dash-core-components
    - dash-html-components
    - dash-renderer


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    brotli-python-1.0.9        |   py39he80948d_5         352 KB  conda-forge
    certifi-2021.10.8          |   py39hf3d152e_2         145 KB  conda-forge
    dash-2.3.1                 |     pyhd8ed1ab_0         7.5 MB  conda-forge
    dash-core-components-2.0.0 |     pyhd8ed1ab_1           7 KB  conda-forge
    dash-html-components-2.0.0 |     pyhd8ed1ab_1           8 KB  conda-forge
    dash-renderer-1.9.1        |     pyhd8ed1ab_0         807 KB  conda-forge
    flask-2.1.1                |     pyhd8ed1ab_0          70 KB  conda-forge
    flask-compress-1.11        |     pyhd8ed1ab_0          14 KB  conda-forge
    itsdangerous-2.1.

In [34]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px


spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


app = dash.Dash(__name__)

uniquelaunchsites = spacex_df['Launch Site'].unique().tolist()
lsites = []
lsites.append({'label': 'All Sites', 'value': 'All Sites'})
for site in uniquelaunchsites:
 lsites.append({'label': site, 'value': site})                                 
                           


app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                
                                

                    
                                dcc.Dropdown(id='site_dropdown',options=lsites,placeholder='Select a Launch Site here', searchable = True , value = 'All Sites'),
                                html.Br(),

                       
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                             
                                dcc.RangeSlider(
                                    id='payload_slider',
                                    min=0,
                                    max=10000,
                                    step=1000,  
                                    marks = {
                                            0: '0 kg',
                                            1000: '1000 kg',
                                            2000: '2000 kg',
                                            3000: '3000 kg',
                                            4000: '4000 kg',
                                            5000: '5000 kg',
                                            6000: '6000 kg',  
                                            7000: '7000 kg',
                                            8000: '8000 kg',
                                            9000: '9000 kg',
                                            10000: '10000 kg'                                         
                                    },

                                    value=[min_payload,max_payload]
                                ),
                            
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                              
                                ])

@app.callback(
     Output(component_id='success-pie-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value')]               
)
def update_graph(site_dropdown):
    if (site_dropdown == 'All Sites'):
        df  = spacex_df[spacex_df['class'] == 1]
        fig = px.pie(df, names = 'Launch Site',hole=.3,title = 'Total Success Launches By all sites')
    else:
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        fig = px.pie(df, names = 'class',hole=.3,title = 'Total Success Launches for site '+site_dropdown)
    return fig

@app.callback(
     Output(component_id='success-payload-scatter-chart',component_property='figure'),
     [Input(component_id='site_dropdown',component_property='value'),Input(component_id="payload_slider", component_property="value")]               
)
def update_scattergraph(site_dropdown,payload_slider):
    if site_dropdown == 'All Sites':
        low, high = payload_slider
        df  = spacex_df
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class", 
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    else:
        low, high = payload_slider
        df  = spacex_df.loc[spacex_df['Launch Site'] == site_dropdown]
        mask = (df['Payload Mass (kg)'] > low) & (df['Payload Mass (kg)'] < high)
        fig = px.scatter(
            df[mask], x="Payload Mass (kg)", y="class", 
            color="Booster Version",
            size='Payload Mass (kg)',
            hover_data=['Payload Mass (kg)'])
    return fig


if __name__ == '__main__':
    app.run_server()